In [1]:
import torch
torch.__version__

'1.13.1'

In [5]:
checkpoint_path = 'parcnet/pretrained_models/detection/checkpoint_ema_avg.pt'


exception = [
    'ssd_heads.0.proj_layer.block.conv.weight',
    'ssd_heads.0.loc_cls_layer.dw_conv.block.conv.weight',
    'ssd_heads.1.proj_layer.block.conv.weight',
    'ssd_heads.1.loc_cls_layer.dw_conv.block.conv.weight',
    'ssd_heads.2.loc_cls_layer.dw_conv.block.conv.weight',
    'ssd_heads.3.loc_cls_layer.dw_conv.block.conv.weight',
    'ssd_heads.4.loc_cls_layer.dw_conv.block.conv.weight'
]

checkpoint = torch.load(checkpoint_path, map_location='cpu')
for key in checkpoint.keys():
    if key in exception:
        continue
    elif key.startswith('ssd_heads.5') and key.endswith('conv.weight'):
        # print(key)
        # print(checkpoint[key].shape)
        weight_tens = checkpoint[key]
        checkpoint[key] = weight_tens[:28, :, :, :]
    elif key.startswith('ssd_heads.5') and key.endswith('conv.bias'):
        # print(key)
        # print(checkpoint[key].shape)
        bias_tens = checkpoint[key]
        checkpoint[key] = bias_tens[:28]
    elif key.startswith('ssd_heads') and key.endswith('conv.weight'):
        # print(key)
        # print(checkpoint[key].shape)
        weight_tens = checkpoint[key]
        checkpoint[key] = weight_tens[:42, :, :, :]
    elif key.startswith('ssd_heads') and key.endswith('conv.bias'):
        # print(key)
        # print(checkpoint[key].shape)
        bias_tens = checkpoint[key]
        checkpoint[key] = bias_tens[:42]

torch.save(checkpoint, 'parcnet/pretrained_models/detection/checkpoint_ema_avg_num3.pt')

In [ ]:
from ultralytics import YOLO
import numpy as np
from PIL import Image, ImageDraw

print(np.__version__, np.__path__)
np.bool(False)

In [4]:
model = YOLO("pretrained/yolov8_krsbi.pt")


In [ ]:
CLASSES = ['robot', 'ball', 'goal']
    
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))
COLORS = COLORS.astype(np.uint8)

img = 'images_test/krsbi5.jpg'
results = model(img)

image = Image.open(img)
draw = ImageDraw.Draw(image)

boxes = results[0].boxes

for box, score, idx in zip(boxes, boxes.conf, boxes.cls):
    # score = boxes.conf
    # print(score)
    # idx = boxes.cls
    # print(idx)
    (startX, startY, endX, endY) = box.xyxy.cpu().numpy().squeeze()
    idx = int(idx.item())
    score = score.item()
    label = "{}: {:.2f}%".format(CLASSES[idx], score * 100)

    draw.rectangle(
        [(startX, startY), (endX, endY)],
        outline=tuple(COLORS[idx]),
        width=3
    )
    y = startY - 15 if startY - 15 > 15 else startY + 15
    draw.text((startX, y), label, tuple(COLORS[idx]))

image

In [3]:
model.export(format='coreml')

Ultralytics YOLOv8.0.38 🚀 Python-3.9.15 torch-1.12.1 CPU
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients, 257.4 GFLOPs

PyTorch: starting from pretrained/yolov8_krsbi.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (130.4 MB)

CoreML: starting export with coremltools 6.1...
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 979/979 [00:53<00:00, 18.27 ops/s]
CoreML: export success ✅ 673.6s, saved as pretrained/yolov8_krsbi.mlmodel (260.0 MB)

Export complete (680.7s)
Results saved to /Users/eky/Documents/_SKRIPSI/code/pretrained
Predict:         yolo task=detect mode=predict model=pretrained/yolov8_krsbi.mlmodel imgsz=640
Validate:        yolo task=detect mode=val model=pretrained/yolov8_krsbi.mlmodel imgsz=640 data=krsbi.yaml 
Visualize:       https://netron.app


'pretrained/yolov8_krsbi.mlmodel'